## Structural Output

Use cases for structured outputs include:

* Parsing data from documents
* Extracting data from images
* Structuring all language model responses
* More reliability and consistency than JSON mode

#### Example 1

In [1]:
from typing import List, Literal, Optional
from ollama import chat
from pydantic import BaseModel

class Equipment(BaseModel):
    jobtype: str
    materials: list[str]

response = chat(
    messages=[
        {
        'role': 'user',
        'content': """Airmaster is pleased to provide you with this proposal to replace and repair equipment at 108 North Terrace, including the AHU_03 Chilled Water Valve Actuator in Plantroom 2, serving Mall West, CHWP_09 Contactor in Chiller Plantroom, CT_09 Inlet Float Valve in NW Chiller Plantroom, and AHU_10 Air Filter Magnehelic Gauge in Plantroom 1.

                    The proposed works involve replacing faulty components, including valve actuators, contactors, float valves, and magnehelic gauges. These parts play crucial roles in operating the air handling units, chilled water pumps, and cooling towers. Technician's notes indicate that AHU_03 cannot operate reliably until the actuator is replaced, and replacing CHWP_09's contactor will prevent a potential failure affecting occupants' thermal comfort.

                    A detailed scope of work and the price are provided below for consideration."""
         }
         ],
         model='llama3.1',
         format= Equipment.model_json_schema()
 )

equipment = Equipment.model_validate_json(response.message.content)
print(equipment)


jobtype='Commercial Equipment Maintenance' materials=['Valve Actuators', 'Contactors', 'Float Valves', 'Magnehelic Gauges']


#### Example 2

In [2]:
class Pet(BaseModel):
    name: str
    animal: int
    age: int 
    color: str | None 
    favorite_toy: str | None 

class PetList(BaseModel):
    pets: list[Pet]

response = chat(
    messages=[
        {
          'role': 'user',
          'content':"""
                    I have two pets.
        A cat named Luna who is 5 years old and loves playing with yarn. She has grey fur.
        I also have a 2 year old black cat named Loki who loves tennis balls.
        """
    }
 ],
 model='llama3.1',
 format=PetList.model_json_schema()
)

pets = PetList.model_validate_json(response.message.content)
print(pets)

pets=[Pet(name='Luna', animal=4, age=5, color='grey', favorite_toy='yarn'), Pet(name='Loki', animal=4, age=2, color='black', favorite_toy='tennis balls')]


#### Example 3: Openai Compatibility

In [6]:
from openai import OpenAI
import openai

client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

try:
    completion = client.beta.chat.completions.parse(
        temperature=0,
        model="llama3.1:latest",
        messages=[
            {
                "role": "user",
                "content": "I have two pets. A cat named Luna who is 5 years old and loves playing with yarn. She has grey fur. I also have a 2 year old black cat named Loki who loves tennis balls."
            }
        ],

    )
    pet_response = completion.choices[0].message
    if pet_response.parsed:
        print(pet_response.parsed)
    elif pet_response.refusal:
        print(pet_response.refusal)
except Exception as e:
    if type(e) == openai.LengthFinishedError:
        print("Max tokens reached: ", e)
        # pass
    else:
        print("Error: ", e)
        # pass

#### 4. Image Descriptions

In [9]:
class Object(BaseModel):
    name: str
    confidence: float 
    attribute: str

class ImageDescription(BaseModel):
    summary: str 
    objects: List[Object]
    scene: str
    colors: List[str]
    time_of_day: Literal['Morning', 'Afternoon', 'Evening', 'Night']
    setting: Literal['Indoor', 'Outdoor', 'Unknown']
    text_content: Optional[str] = None

path = 'images/man_climbing_tree.png'

response = chat(
    model='llama3.2-vision',
    format=ImageDescription.model_json_schema(), # Pass in the schema for the response 
    messages=[ {
        'role': 'user',
        'content': 'Analyze this image and describe what you see, including any objects, the scene, colors and any text you can detect.',
      'images': [path],
    } 
    ], 
    options={'temperature': 0}, # Set the temperature to 0 for deterministic results

)

image_description = ImageDescription.model_validate_json(response.message.content)
print(image_description)

summary='A man is climbing a tree to pick apples.' objects=[Object(name='man', confidence=0.9, attribute='climbing a tree'), Object(name='tree', confidence=1.0, attribute='with branches and leaves'), Object(name='apples', confidence=1.0, attribute='red and ripe')] scene='a man is climbing a tree to pick apples.' colors=['green', 'brown', 'red'] time_of_day='Afternoon' setting='Outdoor' text_content='The man is picking apples from the tree.'


For reliable use of structured outputs, consider to:

* Use Pydantic (Python) or Zod (JavaScript) to define the schema for the response
* Add “return as JSON” to the prompt to help the model understand the request
* Set the temperature to 0 for more deterministic output